In [4]:
import pandas as pd
import csv
import glob
import re
import os
from pathlib import Path

def find_latest_file(pattern, directory):
    """
    Hittar den senaste filen baserat på datum i filnamnet.
    Pattern ska vara t.ex. 'product_sales_*.csv' eller 'stock_report_*.csv'
    """
    directory = Path(directory)
    
    # Kontrollera att katalogen finns
    if not directory.exists():
        raise FileNotFoundError(f"Katalogen finns inte: {directory}")
    
    files = glob.glob(str(directory / pattern))
    if not files:
        # Visa vad som faktiskt finns i katalogen för debugging
        existing_files = list(directory.glob('*.csv'))
        if existing_files:
            print(f"  Hittade {len(existing_files)} CSV-filer i {directory}, men inga matchar mönstret {pattern}")
            print(f"  Exempel filer: {[f.name for f in existing_files[:5]]}")
        else:
            print(f"  Katalogen {directory} är tom eller innehåller inga CSV-filer")
        raise FileNotFoundError(f"Inga filer matchar mönstret {pattern} i {directory}")
    
    # Extrahera datum från filnamn och sortera
    file_dates = []
    for file in files:
        # Försök hitta datum i formatet YYYY-MM-DD eller YYYY-MM-DD_to_YYYY-MM-DD
        date_match = re.search(r'(\d{4}-\d{2}-\d{2})', os.path.basename(file))
        if date_match:
            # Om det finns två datum (t.ex. 2024-02-12_to_2025-12-09), ta det sista
            all_dates = re.findall(r'(\d{4}-\d{2}-\d{2})', os.path.basename(file))
            if all_dates:
                latest_date_in_filename = max(all_dates)
                file_dates.append((latest_date_in_filename, file))
    
    if not file_dates:
        raise ValueError(f"Kunde inte hitta datum i filnamn för {pattern}")
    
    # Sortera efter datum (senaste först)
    file_dates.sort(key=lambda x: x[0], reverse=True)
    latest_file = file_dates[0][1]
    print(f"  Hittade senaste fil: {os.path.basename(latest_file)} (datum: {file_dates[0][0]})")
    return latest_file

# Läs in parametrar/Beställningsfrekvens.csv manuellt för att undvika BOM-problem
# Använd utf-8-sig för att automatiskt ta bort BOM-tecknet
rows = []
with open('data/parametrar/Beställningsfrekvens.csv', 'r', encoding='utf-8-sig') as f:
    reader = csv.reader(f, delimiter=';')
    for row in reader:
        rows.append(row)

# Första raden är header, resten är data
header = rows[0]
data_rows = rows[1:]

# Extrahera leverantörer från kolumn 0 (första kolumnen)
leverantorer_frekvens = set()
for row in data_rows:
    if len(row) > 0:
        leverantor = row[0].strip()
        if leverantor:  # Hoppa över tomma rader
            leverantorer_frekvens.add(leverantor)

# Hitta senaste product_sales_items-filen dynamiskt
print("Hittar senaste product_sales_items-fil...")
latest_items_file = find_latest_file('product_sales_items_*.csv', 'data/downloads')

# Läs in product_sales_items-filen
df_products = pd.read_csv(
    latest_items_file, 
    encoding='utf-8'
)

# Unika leverantörsnamn från product_sales_items
leverantorer_products = set(df_products['supplier_name'].dropna().astype(str).str.strip())

# Hur många unika leverantörer i respektive fil?
n_frekvens = len(leverantorer_frekvens)
n_products = len(leverantorer_products)

# Direkta matches
direkta_matches = leverantorer_frekvens & leverantorer_products
n_matches = len(direkta_matches)

print(f"Antal unika leverantörer i parametrar/Beställningsfrekvens.csv: {n_frekvens}")
print(f"Antal unika leverantörer i product_sales_items: {n_products}")
print(f"Antal direkta matches mellan filerna: {n_matches}")
print("\nExempel på direkta matches (max 10):")
print(list(direkta_matches)[:10])

# Visa alla leverantörer från parametrar/Beställningsfrekvens.csv
print(f"\nAlla {n_frekvens} leverantörer i parametrar/Beställningsfrekvens.csv:")
for i, lev in enumerate(sorted(leverantorer_frekvens), 1):
    print(f"  {i:2d}. {lev}")

# Visa alla leverantörer från product_sales_items
print(f"\nAlla {n_products} leverantörer i product_sales_items:")
for i, lev in enumerate(sorted(leverantorer_products), 1):
    print(f"  {i:2d}. {lev}")


Antal unika leverantörer i parametrar/Beställningsfrekvens.csv: 35
Antal unika leverantörer i product_sales_items: 52
Antal direkta matches mellan filerna: 11

Exempel på direkta matches (max 10):
['Eggpack', 'Snellman', 'Fresh Servant', 'Torp Handelsträdgård', 'Närpes glass', 'Granholms', 'Kållby honung', 'Österbacka', 'Boströms Bageri', 'Kespro']

Alla 35 leverantörer i parametrar/Beställningsfrekvens.csv:
   1. Arla
   2. Bergwall
   3. Boströms Bageri
   4. Chocolateierane
   5. Dahlbo must
   6. Eggpack
   7. Ekofish Ab
   8. Fagerudd bageri
   9. Finnspring
  10. Fresh Servant
  11. Granholm sylt
  12. Granholms
  13. Häggbloms potatis
  14. Jeriko gård
  15. Kespro
  16. Korv-Görans Kebab Ab Oy
  17. Kp Finberry Oy
  18. Kållby honung
  19. Lasse Forsman Gourmet garage
  20. Liselunds must
  21. Luukkonens bageri
  22. Marcs gård
  23. Metro pikatukku (Wihuri)
  24. Närpes glass
  25. Otto Gård/ mjöl
  26. SIA Glass
  27. Snellman
  28. Sunds trädgård
  29. Texet products
  30. 

In [5]:
# Skapa en lista med alla unika stores från kolumnen "store_name"
stores = sorted(df_products['store_name'].dropna().astype(str).str.strip().unique())
print("Alla stores i alfabetisk ordning:")
for i, store in enumerate(stores, 1):
    print(f"{i:2d}. {store}")


Alla stores i alfabetisk ordning:
 1. Bosund kiosk
 2. Granholmen kiosk
 3. Holm kiosk
 4. Kronoby kiosk
 5. Lager kållby
 6. Sisbacka kiosk
 7. Sunds kiosk
 8. Torp kiosk
 9. V-lift kiosk
